In [1]:
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GPT2LMHeadModel, GPT2Tokenizer
import re
import importlib


In [2]:
tokenizer_T5 = AutoTokenizer.from_pretrained("google/flan-t5-base")
model_T5 = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base",
                                                 pad_token_id=tokenizer_T5.eos_token_id)

tokenizer_GPT2 = GPT2Tokenizer.from_pretrained("gpt2")
model_GPT2 = GPT2LMHeadModel.from_pretrained("gpt2",
                                             pad_token_id=tokenizer_GPT2.eos_token_id)


In [3]:
from preprocessing import process_stimuli
from llm_inference import get_response
import llm_inference
# from postprocessing import count_string_occurrences

In [5]:
from preprocessing import convert_exp1
importlib.reload(llm_inference)

def join_prompts(thinks, announced):

    gen_explanation = 'Consider the two following sentences and fill in the values <<< and >>> ' \
                  'in the sentence, in which the value would be most likely.\n'
    ending = 'Answer with the number for sentence 1. first and then with the number for sentence 2. '
    prompts = []
    for t, a in zip(thinks, announced):
        #
        # separate sentences
        sentences = re.split(r'\. ', t)
        description = sentences[0] + '. ' + sentences[1] + '. '
        split_t = re.split(r'XXX', sentences[2])
        numbers = re.findall(r'\d+', split_t[1])
        explanation = re.sub('<<<', numbers[0], gen_explanation)
        explanation = re.sub('>>>', numbers[1], explanation)
        # t_sentence = '1. ' + split_t[0] + '\n'
        t_sentence = split_t[0] + '\n'


        # announced condition
        a_sentences = re.split(r'\. ', a)
        split_a = re.split(r'XXX', a_sentences[2])
        # a_sentence = '2. ' + split_a[0] + '\n'
        a_sentence = split_a[0] + '\n'





        prompt = description + explanation + t_sentence + a_sentence + ending
        prompts.append(prompt)
        # print(prompt)
    return prompts

#get the prompts for the 2 conditions in 2 lists
thinks, announced = process_stimuli("stimuli/stimuli_Exp1.txt")
prompts = join_prompts(thinks, announced)
#print(prompts)

#get the full responses to look at
results = llm_inference.get_response(model_T5, tokenizer_T5, n_repetitions=1, prompts=prompts, condition='both sentences')
 # think_resp_counts = count_string_occurrences(think_response, "Number")
# announce_resp_counts = count_string_occurrences(announce_response, "Number")'(\. ', t)
        #description = sentences[0] + ' .' + sentences[1] + '. '

Joseph is a man from the US. Joseph lives next door to Sue. Consider the two following sentences and fill in the values 8 and 11 in the sentence, in which the value would be most likely.
Sue thinks that Joseph ate ___ burgers last month
Sue announced to me that Joseph ate ___ burgers last month
Answer with the number for sentence 1. first and then with the number for sentence 2. 
8
Nick is a man from the US. Nick went to school with Stephanie. Consider the two following sentences and fill in the values 22 and 36 in the sentence, in which the value would be most likely.
Stephanie thinks that Nick saw __ movies last year
Stephanie announced to me that Nick saw ___ movies last year
Answer with the number for sentence 1. first and then with the number for sentence 2. 
3
Melanie is a woman from the US. Melanie has a colleague, Bob. Consider the two following sentences and fill in the values 73 and 152 in the sentence, in which the value would be most likely.
Bob thinks that Melanie owns ___

In [5]:
results

,Condition,Prompt,Response
0,both sentences,Joseph is a man from the US. Joseph lives next...,4
1,both sentences,Nick is a man from the US. Nick went to school...,3
2,both sentences,Melanie is a woman from the US. Melanie has a ...,73
3,both sentences,Erin is a first grade student in primary schoo...,13
4,both sentences,Betty is a woman from the US. Betty works at a...,27
5,both sentences,Lelia is a woman from the US. Lelia lives arou...,2
6,both sentences,"Tony is a man from the US. Nick has a sister, ...",4
7,both sentences,Liam is a man from the US. Liam lives down the...,29
8,both sentences,Judith is a woman from the US. Judith has a br...,268
9,both sentences,"Andy is a man from the US. Andy has an aunt, K...",3


In [6]:
results = llm_inference.get_response(model_GPT2, tokenizer_GPT2, n_repetitions=1, prompts=prompts, condition='both sentences')

Joseph is a man from the US. Joseph lives next door to Sue. Consider the two following sentences and fill in the values 8 and 11 in the sentence, in which the value would be most likely.
Sue thinks that Joseph ate ___ burgers last month
Sue announced to me that Joseph ate ___ burgers last month
 
Joseph is a man from the US. Joseph lives next door to Sue. Consider the two following sentences and fill in the values 8 and 11 in the sentence, in which the value would be most likely.
Sue thinks that Joseph ate ___ burgers last month
Sue announced to me that Joseph ate ___ burgers last month
  I am not sure if Joseph ate ___ burgers last month or not, but he probably ate sushi last year.
Sue wears a baseball cap and wears a baseball jacket.
Sue doesn't really wear baseball cap and jacket, but she wears a baseball cap and jacket.
Sue is wearing a baseball cap and jacket  and cell phone into a car.
Sue has a child.
Sue is married.
Sue seems to be a gamer.
In fact, I doubt that she is a gamer.

In [10]:
from llm_inference import generate
response = generate('hello i am a duck, what are you?'
                    '', model_T5, tokenizer_T5)
response

'A duck in a duck suit.'

In [ ]:
from fitbert import FitBert

def convert_for_bert(text):
    text = text.replace("YYYY", "XXX")
    text = text.replace("__", "***mask***")
    list = text.split("XXX")
    for i in range(len(list)):
        list[i] = list[i].strip()
    prompt = list[0]
    options = list[1:]
    return prompt, options

def fitbert_ranking(sentence, alternatives, model):
    return model.rank(sentence, options=alternatives, with_prob=True)

fb = FitBert()

counts = [0,0] # [lower_count, higher_count], assuming that the lower comes first in the exercise
for p in range(1): # because there were 90 participants; in the original study, participants were treated separately
    for tq in low:
        result = fitbert_ranking(tq[0], tq[1], fb)
        print(result[0][0])
        print(tq[1][1])
        if result[0][0] == tq[1][0].strip(): # das besser oben implementieren
            counts[0]+=1
        elif result[0][0] == tq[1][1].strip():
            counts[1]+=1